# Aligning Sentinel-2 and Sentinel-3 OLCI Data

You can find the Colab version of this notebook [here](https://drive.google.com/drive/folders/1-aQbcIaohFr4PzzkuCd3k_IXy_Yjw8QP?usp=sharing).In our previous session, we explored the method of retrieving colocated Sentinel-2 optical data alongside Sentinel-3 OLCI data. Building upon that foundation, this week's focus shifts towards the detailed processing and precise alignment of these datasets. Our ultimate goal is to achieve pixel-level colocation between the datasets, enhancing the accuracy and utility of our analysis.

## Step 0: Importing Sentinel-2 and Sentinel-3 OLCI Data

At this juncture, we reintroduce some previously utilised code with a pivotal objective: to import raw data and transform it into a format amenable to our analysis requirements. Given the extensive memory demands of this operation, we recommend a conceptual understanding of the process rather than direct execution. This step serves as a critical foundation, ensuring that subsequent analyses are built on a robust and standardised dataset. 

In [ ]:
! pip install rasterio
! pip install netCDF4

In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import netCDF4
import numpy as np
import re

# Define the path to the main folder where your data is stored.
# You need to replace 'path/to/data' with the actual path to your data folder.
main_folder_path = '/content/drive/MyDrive/GEOL0069/Assignment_Material'
# main_folder_path = './'
# This part of the code is responsible for finding all directories in the main_folder that end with '.SEN3'.
# '.SEN3' is the format of the folder containing specific satellite data files (in this case, OLCI data files).
directories = [d for d in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, d)) and d.endswith('002.SEN3')] #load OLCI imagery

# Loop over each directory (i.e., each set of data) found above.
for directory in directories:
    # Construct the path to the OLCI data file within the directory.
    # This path is used to access the data files.
    OLCI_file_p = os.path.join(main_folder_path, directory)

    # Print the path to the current data file being processed.
    # This is helpful for tracking which file is being processed at any time.
    print(f"Processing: {OLCI_file_p}")

    # Load the instrument data from a file named 'instrument_data.nc' inside the directory.
    # This file contains various data about the instrument that captured the satellite data.
    instrument_data = netCDF4.Dataset(OLCI_file_p + '/instrument_data.nc')
    solar_flux = instrument_data.variables['solar_flux'][:]  # Extract the solar flux data.
    detector_index = instrument_data.variables['detector_index'][:]  # Extract the detector index.

    # Load tie geometries from a file named 'tie_geometries.nc'.
    # Tie geometries contain information about viewing angles, which are important for data analysis.
    tie_geometries = netCDF4.Dataset(OLCI_file_p + '/tie_geometries.nc')
    SZA = tie_geometries.variables['SZA'][:]  # Extract the Solar Zenith Angle (SZA).

    # Create a directory for saving the processed data using the original directory name.
    # This directory will be used to store output files.
    save_directory = os.path.join('path/to/save', directory)
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # This loop processes each radiance band in the OLCI data.
    # OLCI instruments capture multiple bands, each representing different wavelengths.
    OLCI_data = []
    for Radiance in range(1, 22):  # There are 21 bands in OLCI data.
    # for Radiance in [2,5,8,16]:  # selecting relevant bands

        Rstr = "%02d" % Radiance  # Formatting the band number.
        solar_flux_band = solar_flux[Radiance - 1]  # Get the solar flux for the current band.

        # Print information about the current band being processed.
        # This includes the band number and its corresponding solar flux.
        print(f"Processing Band: {Rstr}")
        print(f"Solar Flux for Band {Rstr}: {solar_flux_band}")

        # Load radiance values from the OLCI data file for the current band.
        OLCI_nc = netCDF4.Dataset(OLCI_file_p + '/Oa' + Rstr + '_radiance.nc')
        radiance_values = np.asarray(OLCI_nc['Oa' + Rstr + '_radiance'])

        # Initialize an array to store angle data, which will be calculated based on SZA.
        angle = np.zeros_like(radiance_values)
        for x in range(angle.shape[1]):
            angle[:, x] = SZA[:, int(x/64)]

        # Calculate the Top of Atmosphere Bidirectional Reflectance Factor (TOA BRF) for the current band.
        TOA_BRF = (np.pi * radiance_values) / (solar_flux_band[detector_index] * np.cos(np.radians(angle)))

        # Add the calculated TOA BRF data to the OLCI_data list.
        OLCI_data.append(TOA_BRF)

    reshaped_array = np.moveaxis(np.array(OLCI_data), 0, -1)
    OLCI_coord = netCDF4.Dataset(OLCI_file_p + '/geo_coordinates.nc')
    OLCI_lon=OLCI_coord['longitude']
    OLCI_lat=OLCI_coord['latitude']


In [ ]:
#This takes ~60 seconds and ~2Gb RAM

import rasterio
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

# Paths to the band images
path = '' # You need to specify the path
base_path = "S2A_MSIL1C_20190301T235611_N0207_R116_T01WCU_20190302T014622.SAFE/GRANULE/L1C_T01WCU_A019275_20190301T235610/IMG_DATA/"
bands_paths = {
    'B4': path + base_path + 'T01WCU_20190301T235611_B04.jp2',
    'B3': path + base_path + 'T01WCU_20190301T235611_B03.jp2',
    'B2': path + base_path + 'T01WCU_20190301T235611_B02.jp2'
}

# Read and stack the band images
band_data = []
for band in ['B4', 'B3', 'B2']:
    with rasterio.open(bands_paths[band]) as src:
        band_data.append(src.read(1))

# Stack bands and create a mask for valid data (non-zero values in all bands)
band_stack = np.dstack(band_data)
valid_data_mask = np.all(band_stack > 0, axis=2)

# Reshape for GMM, only including valid data
X = band_stack[valid_data_mask].reshape((-1, 3))

s2_filename = path + './S2A_MSIL1C_20190301T235611_N0207_R116_T01WCU_20190302T014622.SAFE/GRANULE/L1C_T01WCU_A019275_20190301T235610/IMG_DATA/T01WCU_20190301T235611_B04.jp2'

# Read the Sentinel-2 image and its geospatial information
with rasterio.open(s2_filename) as src:
    # Read the raster data and the affine transformation
    s2_data = src.read(1)
    transform_matrix = src.transform

    # Get the spatial reference system (CRS)
    srs = src.crs

# Create grid of X,Y values
rows, cols = s2_data.shape
x_s2, y_s2 = [], []
for row in range(rows):
    print(row)
    for col in range(cols):
        x, y = transform_matrix * (col, row)
        x_s2.append(x)
        y_s2.append(y)

# Convert grid of X,Y values to latitude/longitude
in_proj = Proj(init=str(srs))  # Initialize projection from CRS
out_proj = Proj(proj='latlong')  # Initialize projection for latitude/longitude


## Step 1: Loading processed data
As mentioned, you don't need to run the above cells. But you will need to load them from your Google Drive.

In [ ]:
import numpy as np
path = '/content/drive/MyDrive/Teaching_Michel/GEOL0069/StudentFolder/Week_4/' # You need to specify the path
x_s2=np.load(path+'x_s2.npy')
y_s2=np.load(path+'y_s2.npy')
band_stack=np.load(path+'band_stack.npy')

x_s3=np.load(path+'x_s3.npy')
y_s3=np.load(path+'y_s3.npy')
reshaped_array=np.load(path+'reshaped_array.npy')

We can also verify if Sentinel-2 data is indeed within Sentinel-3 OLCI data.

In [ ]:
path = ''
s1=100 #subsampling rate for S2
s2=100 #subsampling rate for s3

plt.scatter(x_s3[::s1,::s1],y_s3[::s1,::s1])
plt.scatter(np.asarray(x_s2).reshape(10980,10980)[::s2,::s2],np.asarray(y_s2).reshape(10980,10980)[::s2,::s2])



In [ ]:
condition = (x_s3 > 360000) & (x_s3 < 380000) & (y_s3 > 7800000) & (y_s3 < 7820000)
plt.scatter(x_s3[condition],y_s3[condition],c=reshaped_array[condition,0],vmin=0.7,vmax=1)
plt.colorbar()
plt.savefig(path+'OLCI_zoom.png',dpi=1200)

#This takes 9 minutes (perhaps saving the figure takes the longest)
x_s2bis=np.asarray(x_s2).reshape(10980,10980)
y_s2bis=np.asarray(y_s2).reshape(10980,10980)
condition = (x_s2bis > 360000) & (x_s2bis < 380000) & (y_s2bis > 7800000) & (y_s2bis < 7820000)
plt.scatter(x_s2bis[condition],y_s2bis[condition],c=band_stack[condition,2]/10000,vmin=0.7,vmax=1)
plt.colorbar()
plt.savefig(path+'S2_zoom.png',dpi=400)


In [ ]:
x_s2_condition = x_s2bis[condition]
y_s2_condition = y_s2bis[condition]
band_stack_condition = band_stack[condition,2]/10000

## Step 2: Overlap optimisation
Now we optimize the overlap between Sentinel-2 image and Senitnel-3 image and better align them.

In [ ]:
reshaped_array_rolled = np.roll(reshaped_array, 3, axis=0)
# Roll x_s3_rolled array by -5 grids in the y-direction
reshaped_array_rolled = np.roll(reshaped_array_rolled, -5, axis=1)

In [ ]:
#Code that interpolates on a common grid before doing matcing of images in terms of colours and correlating to find offset


import numpy as np
from scipy.signal import correlate2d
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from skimage import exposure  # Import exposure module from skimage

# Scatter input values
condition = (x_s3 > 360000) & (x_s3 < 380000) & (y_s3 > 7800000) & (y_s3 < 7820000)
x_s3_condition, y_s3_condition = x_s3[condition], y_s3[condition]
# reshaped_array_condition = reshaped_array[condition, 0]
reshaped_array_condition = reshaped_array_rolled[condition, 0]
# x_s2_condition, y_s2_condition, band_stack_condition = x_s2[condition], y_s2[condition], band_stack[condition]

# Define grid coordinates
ngrid=400 #here it means grid is 20000/400=50m
x_grid = np.linspace(min(x_s3_condition.min(), x_s2_condition.min()), max(x_s3_condition.max(), x_s2_condition.max()), ngrid)
y_grid = np.linspace(min(y_s3_condition.min(), y_s2_condition.min()), max(y_s3_condition.max(), y_s2_condition.max()), ngrid)
x_grid, y_grid = np.meshgrid(x_grid, y_grid)

# Interpolate scattered values onto grid
ss=10 #only use every 10th point of S2 images
z1_grid = griddata((x_s2_condition[::ss], y_s2_condition[::ss]), band_stack_condition[::ss], (x_grid, y_grid), method='cubic')
z2_grid = griddata((x_s3_condition, y_s3_condition), reshaped_array_condition, (x_grid, y_grid), method='cubic')


# Perform histogram matching
matched_z2_grid = exposure.match_histograms(z2_grid, z1_grid)

# Find the translation that optimizes overlap using cross-correlation
z1_grid[z1_grid>0.75]=1
z1_grid[z1_grid<0.75]=0.7
z2_grid[z2_grid>0.75]=1
z2_grid[z2_grid<0.75]=0.7
matched_z2_grid[matched_z2_grid>0.75]=1
matched_z2_grid[matched_z2_grid<0.75]=0.7

#Without the binary classification above I have found that the correlation doesn't work to find alignment
z1_grid_no_nan = np.nan_to_num(z1_grid, nan=1)
matched_z2_grid_no_nan = np.nan_to_num(matched_z2_grid, nan=1)
corr = correlate2d(z1_grid_no_nan, matched_z2_grid_no_nan, mode='same', boundary='wrap')  # Compute cross-correlation
y, x = np.unravel_index(np.argmax(corr), corr.shape)  # Find the indices of the maximum correlation
dx = x - matched_z2_grid.shape[1] // 2  # Compute the x-offset
dy = y - matched_z2_grid.shape[0] // 2  # Compute the y-offset
matched_z2_grid_aligned = np.roll(np.roll(matched_z2_grid_no_nan, dy, axis=0), dx, axis=1)


print("Optimal translation (dx, dy):", dx, dy)


In [ ]:
matched_z2_grid_aligned.shape
plt.pcolor(z1_grid_no_nan,vmin=0.7,vmax=1)
plt.savefig(path+'S2_binary_grid_50m.png',dpi=400)
plt.show()

plt.pcolor(matched_z2_grid_aligned,vmin=0.7,vmax=1)
plt.savefig(path+'S3_binary_matched_S2_grid_50m.png',dpi=400)
plt.show()

The below code does colocation within grid:

In [ ]:
from scipy.spatial import cKDTree

condition = (x_s3 > 360000) & (x_s3 < 380000) & (y_s3 > 7800000) & (y_s3 < 7820000)
x_s3_condition, y_s3_condition = x_s3[condition], y_s3[condition]

# Define a KD-tree using x_s2_condition and y_s2_condition
tree = cKDTree(list(zip(x_s2_condition, y_s2_condition)))

# Query the tree to find all points within x_s3_condition and y_s3_condition grids (Choose one of the following 2 methods)
# Option 1:
indices_within_grid = tree.query_ball_point(list(zip(x_s3_condition, y_s3_condition)), r=300.0)
# Option 2: Query for a square neighbourhood
max_radius=1000
indices_within_grid = tree.query(list(zip(x_s3_condition[::100], y_s3_condition[::100])), k=len(x_s2_condition), p=40, distance_upper_bound=max_radius)[1]

In [ ]:
# Plot one colocated S2 data for one S3 cell (Within a radius of 300m)
plt.scatter(x_s2_condition[indices_within_grid[10][0:500]],y_s2_condition[indices_within_grid[10][0:500]],c=band_stack_condition[indices_within_grid[0][0:500]]/10000)#,vmin=0.7,vmax=1)